<a href="https://colab.research.google.com/github/AishaEvering/LLM_Mastery/blob/main/Masked_Language_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from scipy.special import softmax
import numpy as np

In [ ]:
MODEL_NAME = 'bert-base-cased'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
model = AutoModelForMaskedLM.from_pretrained(MODEL_NAME)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
mask = tokenizer.mask_token
mask

'[MASK]'

In [ ]:
sentence = f'I want to {mask} pizza for tonight.'
sentence2 = f'I {mask} my keys.'

In [ ]:
tokens = tokenizer.tokenize(sentence)
tokens

['I', 'want', 'to', '[MASK]', 'pizza', 'for', 'tonight', '.']

In [ ]:
encoded_inputs = tokenizer(sentence, return_tensors='pt')
encoded_inputs

{'input_ids': tensor([[  101,   146,  1328,  1106,   103, 13473,  1111,  3568,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
outputs = model(**encoded_inputs)
outputs

MaskedLMOutput(loss=None, logits=tensor([[[ -7.3723,  -7.2489,  -7.4421,  ...,  -6.3119,  -5.9369,  -6.4257],
         [ -7.9311,  -8.2282,  -8.0326,  ...,  -6.7387,  -6.4877,  -6.9525],
         [-12.0500, -11.7972, -12.5776,  ...,  -8.4518,  -6.7310,  -8.2586],
         ...,
         [-10.2204, -10.4315,  -9.9993,  ...,  -7.9570,  -6.7194,  -9.3618],
         [-12.4471, -12.5367, -12.5614,  ...,  -9.9086,  -9.4219, -11.1770],
         [-14.3657, -14.5227, -15.0017,  ..., -11.9715, -11.6569, -13.4498]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [ ]:
encoded_inputs2 = tokenizer(sentence2, return_tensors='pt')
output2 = model(**encoded_inputs2)

logits2 = output2.logits.detach().numpy()[0]

tokens2 = tokenizer.tokenize(sentence2)

mask_logits2 = logits2[tokens2.index(mask) + 1]

confidence_scores2 = softmax(mask_logits2)

for i in np.argsort(confidence_scores2)[::-1][:15]:
  pred_token = tokenizer.decode(i)
  score = confidence_scores2[i]

  #print(pred_token, score)
  print(sentence2.replace(mask, pred_token), score)

I grabbed my keys. 0.3370596
I grab my keys. 0.18180192
I have my keys. 0.07027481
I found my keys. 0.03430151
I need my keys. 0.031869534
I get my keys. 0.026572486
I got my keys. 0.022518743
I had my keys. 0.02047186
I dropped my keys. 0.015414495
I find my keys. 0.014643141
I checked my keys. 0.01227915
I took my keys. 0.01158159
I retrieved my keys. 0.008924723
I take my keys. 0.008836027
I want my keys. 0.0074018403


In [ ]:
logits = outputs.logits.detach().numpy()[0]

In [ ]:
logits.shape

(10, 28996)

In [ ]:
mask_logits = logits[tokens.index(mask) + 1]
mask_logits

array([-6.714628 , -6.379109 , -6.1184893, ..., -5.651309 , -3.6572778,
       -4.9947314], dtype=float32)

In [ ]:
confidence_scores = softmax(mask_logits)
confidence_scores

array([2.9159888e-10, 4.0784978e-10, 5.2928079e-10, ..., 8.4446000e-10,
       6.2026344e-09, 1.6282734e-09], dtype=float32)

In [ ]:
 confidence_scores.sum()

1.0

In [ ]:
for i in np.argsort(confidence_scores)[::-1][:5]:
  pred_token = tokenizer.decode(i)
  score = confidence_scores[i]

  #print(pred_token, score)
  print(sentence.replace(mask, pred_token), score)

I want to have pizza for tonight. 0.25729063
I want to get pizza for tonight. 0.17849593
I want to eat pizza for tonight. 0.15555556
I want to make pizza for tonight. 0.11422409
I want to order pizza for tonight. 0.0982304
